### <b>SEResNet50

<b> Ref: <br> 
https://arxiv.org/pdf/1709.01507v4.pdf <br>
https://github.com/moskomule/senet.pytorch<br>
https://github.com/Cadene/pretrained-models.pytorch

In [1]:
# !pip install pretrainedmodels

Loading SEResnet-50

In [2]:
import pretrainedmodels

hub_model = pretrainedmodels.se_resnet50(pretrained='imagenet')
# senet = pretrainedmodels.senet154(pretrained='imagenet')

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
from tqdm import tqdm


In [4]:
transform = transforms.Compose([
    transforms.ToTensor()
])


In [5]:
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev) 
print(torch.cuda.get_device_name(device))

NVIDIA GeForce RTX 2060


In [6]:
BATCH_SIZE = 16
TRAIN_DATA_PATH = "C:/Users/gmita/Desktop/work_3_2_2565/onboard/pytorch_ver/Data/train"
VAL_DATA_PATH = "C:/Users/gmita/Desktop/work_3_2_2565/onboard/pytorch_ver/Data/val"
TEST_DATA_PATH = "C:/Users/gmita/Desktop/work_3_2_2565/onboard/pytorch_ver/Data/Test"

In [7]:
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,num_workers=0, shuffle=True,pin_memory=True)
                                    
val_data = torchvision.datasets.ImageFolder(root=VAL_DATA_PATH, transform=transform)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE,num_workers=0,  shuffle=True,pin_memory=True)
                                    
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE,num_workers=0,  shuffle=False,pin_memory=True)

In [8]:
hub_model

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNetBottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_module): SEModule(
        (avg_pool): Ada

In [9]:
#set seed
torch.manual_seed(666)
if torch.cuda.is_available():
    torch.cuda.manual_seed(666)

# set random seed for NumPy
np.random.seed(666)

In [10]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [11]:
feature_extract = True
set_parameter_requires_grad(hub_model, feature_extract)

hub_model.last_linear = nn.Sequential(
          nn.Linear(2048,128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
          nn.Linear(128,4),
          nn.Softmax(dim=1)
        )

In [12]:
params_to_update = hub_model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in hub_model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in hub_model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 last_linear.0.weight
	 last_linear.0.bias
	 last_linear.1.weight
	 last_linear.1.bias
	 last_linear.3.weight
	 last_linear.3.bias


In [13]:
optimizer = optim.SGD(params_to_update, momentum=0.9, lr=0.001)
criterion = nn.functional.cross_entropy

In [14]:
from module import train , testAccuracy

train(30,hub_model,optimizer,criterion,train_loader,val_loader,"SeResNet50.pt")

The model will be running on NVIDIA GeForce RTX 2060 device


Epoch 1/30: 100%|███████████████████████████████████████████████████████████| 88/88 [00:09<00:00,  9.61it/s, Loss=1.26]


For epoch 1, the test accuracy over the whole validation set is 76.0% and loss 1.1201782409961407


Epoch 2/30: 100%|██████████████████████████████████████████████████████████| 88/88 [00:06<00:00, 14.51it/s, Loss=0.989]


For epoch 2, the test accuracy over the whole validation set is 88.5% and loss 0.9024984928277823


...

For epoch 28, the test accuracy over the whole validation set is 98.0% and loss 0.7763650325628427


Epoch 29/30: 100%|█████████████████████████████████████████████████████████| 88/88 [00:06<00:00, 14.65it/s, Loss=0.753]


For epoch 29, the test accuracy over the whole validation set is 98.0% and loss 0.7753041707552396


Epoch 30/30: 100%|█████████████████████████████████████████████████████████| 88/88 [00:06<00:00, 14.54it/s, Loss=0.751]


For epoch 30, the test accuracy over the whole validation set is 98.0% and loss 0.7707409354356619
Training complete!
Total training time: 211.57 seconds


In [15]:

state_dict = torch.load("SeResNet50.pt")
hub_model.load_state_dict(state_dict)

accuracy , loss  = testAccuracy(hub_model,criterion,test_loader)
print(f"Test accuracy = {accuracy}%")


Test accuracy = 95.03722084367246%


In [16]:
import matplotlib.pyplot as plt
import numpy as np

# Set the model to evaluation mode
hub_model.cuda()
hub_model.eval()

# Create a list to store the incorrectly predicted images
wrong_images = []

# Create a list to store the true labels of the incorrectly predicted images
true_labels = []

# Create a list to store the predicted labels of the incorrectly predicted images
pred_labels = []

# for classification report
y_pred = []
y_true = []

# Iterate over the test dataset
for inputs, labels in test_loader:
    # Move the inputs and labels to the device
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = hub_model(inputs)
    _, preds = torch.max(outputs, 1)

    y_pred.extend(preds.cpu().numpy())
    y_true.extend(labels.cpu().numpy())
    # Check if the predicted labels match the true labels
    wrong_idx = torch.where(preds != labels)[0]

    # Add the incorrectly predicted images to the list
    for idx in wrong_idx:
        wrong_images.append(inputs[idx].cpu())
        true_labels.append(labels[idx].cpu().item())
        pred_labels.append(preds[idx].cpu().item())


In [17]:
from sklearn.metrics import classification_report

Class_name = test_loader.dataset.classes
report = classification_report(y_true, y_pred, target_names=Class_name)
print(report)

                           precision    recall  f1-score   support

      Curved Mayo Scissor       0.88      0.93      0.91       104
                  Scalpel       1.00      1.00      1.00       111
Straight Dissection Clamp       1.00      1.00      1.00        83
    Straight Mayo Scissor       0.93      0.88      0.90       105

                 accuracy                           0.95       403
                macro avg       0.95      0.95      0.95       403
             weighted avg       0.95      0.95      0.95       403



<b>summary</b><br>
1. SeResNet50 have more accuracy than normal ResNet50 that have accuracy 0.93 <br>
2. SeResNet50 can perfect classify class 1,2 (Scalpel and Clamp), because the dataset may not be very large.<br>
3. SeResNet50 can classify class 1,2 better than VGG16 <br>
4. Prediction of Class 0,3 (Curved Mayo Scissor and Straight Mayo Scissor) seem is not as good as that of Class 1,2 (Scalpel and Clamp), that it will maybe because of the shape of 2 scissor class look so similar.